In [1]:
import pandas as pd
import numpy as np
import glob
import os
#import arcpy
from io import StringIO
from arcgis.gis import GIS
import sys
import requests

# Python Standard Library Modules
from pathlib import Path
from zipfile import ZipFile

In [2]:
import importlib.util
spec = importlib.util.spec_from_file_location("dbconnect", "G:/My Drive/Python/dbconnect.py")
dbconnect = importlib.util.module_from_spec(spec)
spec.loader.exec_module(dbconnect)

In [3]:
engine = dbconnect.postconn_ugs()

Import the `wqxsde` module (if it is not installed)

In [4]:
wqxpath = "G:/My Drive/Python/Pycharm/wqxsde/"
sys.path.append(wqxpath)
import wqxsde

In [11]:
sdewqx = wqxsde.SDEtoWQX(*dbconnect.postconn_ugs_details(), "G:/My Drive/WORK/Groundwater Chemistry/output/")

In [12]:
sdewqx.ugs_to_upload['Result-Activity']

,activityid,activitymedia,activitystartdate,activitystarttime,activitytimezone,activitytype,analysisstartdate,characteristicname,laboratoryname,methodspeciation,...,resultdetectioncondition,detecquantlimitmeasure,resultdetecquantlimittype,resultdetecquantlimitunit,resultsamplefraction,resultstatusid,resultunit,resultvaluetype,sampcollectionequip,sampcollectmethod
0,1406012-01,Water,2014-06-09,09:20,MDT,Sample-Routine,2014-06-17,Chloride,"U.S. EPA, Region 8 Laboratory",None,...,None,0.20,Method Detection Level,mg/l,Dissolved,Final,mg/l,Actual,Water Bottle,GRAB
1,1406012-01,Water,2014-06-09,09:20,MDT,Sample-Routine,2014-06-17,Nitrate,"U.S. EPA, Region 8 Laboratory",as N,...,Below Detection Limit,5.00,Method Detection Level,ug/l,Dissolved,Final,mg/l,Actual,Water Bottle,GRAB
2,1406012-01,Water,2014-06-09,09:20,MDT,Sample-Routine,2014-06-17,Nitrite,"U.S. EPA, Region 8 Laboratory",as N,...,Below Detection Limit,5.00,Method Detection Level,ug/l,Dissolved,Final,mg/l,Actual,Water Bottle,GRAB
3,1406012-01,Water,2014-06-09,09:20,MDT,Sample-Routine,2014-06-17,Orthophosphate,"U.S. EPA, Region 8 Laboratory",as P,...,None,5.00,Method Detection Level,ug/l,Dissolved,Final,mg/l,Actual,Water Bottle,GRAB
4,1406012-01,Water,2014-06-09,09:20,MDT,Sample-Routine,2014-06-13,Total Dissolved Solids,"U.S. EPA, Region 8 Laboratory",None,...,None,2.00,Method Detection Level,mg/l,Dissolved,Final,mg/l,Actual,Water Bottle,GRAB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11420,1808006-04,Water,2018-08-07,16:15,MDT,Sample-Routine,2018-08-22,Manganese,None,None,...,Below Reporting Limit,2.00,Lower Reporting Limit,ug/l,Total,Final,ug/l,Actual,Water Bottle,GRAB
11421,1808006-04,Water,2018-08-07,16:15,MDT,Sample-Routine,2018-08-22,Copper,None,None,...,Below Reporting Limit,10.00,Lower Reporting Limit,ug/l,Total,Final,ug/l,Actual,Water Bottle,GRAB
11422,1808006-04,Water,2018-08-07,16:15,MDT,Sample-Routine,2018-08-22,Boron,None,None,...,Below Reporting Limit,100.00,Lower Reporting Limit,ug/l,Total,Final,ug/l,Actual,Water Bottle,GRAB
11423,1808006-04,Water,2018-08-07,16:15,MDT,Sample-Routine,2018-08-22,Chromium,None,None,...,Below Reporting Limit,5.00,Lower Reporting Limit,ug/l,Total,Final,ug/l,Actual,Water Bottle,GRAB


In [12]:
sdewqx.ugs_tabs['Activity'].set_index(['activityid'])[['temperature_c','ph','conduct_uscm']].unstack(0).reset_index()

,level_0,activityid,0
0,temperature_c,1406012-01,None
1,temperature_c,1406012-03,None
2,temperature_c,1406012-04,None
3,temperature_c,1406012-05,None
4,temperature_c,1406012-06,None
...,...,...,...
3550,conduct_uscm,1808006-06,None
3551,conduct_uscm,1808006-13,None
3552,conduct_uscm,1808006-13RE1,None
3553,conduct_uscm,1808006-13RE2,None


In [ ]:
sdewqx.wqp_tabs['Activity'].head()

In [ ]:
sdewqx.wqp_tabs['Activity'].head()

In [ ]:
df = sdewqx.results_activities_prep()

In [ ]:
act = sdewqx.ugs_to_upload['Activity'].dropna(axis=1,how='all').drop(['monitoringlocationid'],axis=1)
res = sdewqx.ugs_to_upload['Result'].dropna(axis=1,how='all')

In [ ]:
pd.merge(res,act,on='activityid')

In [ ]:
sdewqx.wqp_tabs['Result'].columns
         

In [ ]:
sdewqx.ugs_to_upload['Result']

In [ ]:
df[(df['monitoringlocationid']=='401043111361801')].sort_values(['activitystartdate'])

In [ ]:
sdewqx.ugs_tabs['Activity']

In [ ]:
pd.merge(sdewqx.ugs_tabs['Result']

In [ ]:
sdewqx.ugs_tabs['Result'][['resultdetectioncondition','detecquantlimitmeasure', 'resultdetecquantlimittype',
                                              'resultdetecquantlimitunit']]

In [ ]:
import datetime
datetime.datetime.today()

In [8]:
chem = 'Arsenic'
def chem_lookup(chem):
    url = f'https://cdxnodengn.epa.gov/cdx-srs-rest/substance/name/{chem}?qualifier=exact'
    rqob = requests.get(url).json()
    moleweight = float(rqob[0]['molecularWeight'])
    moleformula = rqob[0]['molecularFormula']
    casnumber = rqob[0]['currentCasNumber']
    epaname = rqob[0]['epaName']
    return [epaname,moleweight,moleformula,casnumber]
chem_lookup(chem)

['Arsenic', 74.92, 'As', '7440-38-2']

In [ ]:
rqob[0]['subsKey']

In [9]:
#"https://cdxnodengn.epa.gov/cdx-srs-rest/"
char_domains = "http://www.epa.gov/storet/download/DW_domainvalues.xls"
char_schema = pd.read_excel(char_domains)
char_schema = char_schema[['PK_ISN','REGISTRY_NAME','CHARACTERISTIC_GROUP_TYPE','SRS_ID','CAS_NUMBER']]
chemgroups = char_schema[['REGISTRY_NAME', 'CHARACTERISTIC_GROUP_TYPE']].set_index(['REGISTRY_NAME']).to_dict()['CHARACTERISTIC_GROUP_TYPE']


In [ ]:
char_schema[char_schema['CHARACTERISTIC_GROUP_TYPE']=='Inorganics, Major, Metals']

In [ ]:
char_schema.CHARACTERISTIC_GROUP_TYPE.unique()

In [ ]:
requests.get("https://cdxnodengn.epa.gov/cdx-srs-rest/substance/name/Nitrate?qualifier=exact").json()

In [ ]:
wqp = wqxsde.WQP('UTAHGS','organization')

assign the relevant paths to the 

In [ ]:
sde = wqxsde.GetPaths()
arcpy.env.workspace = sde.enviro
wqx_results_filename = "ResultsExport.xlsx"
wqx_stations_filename = "LocationsExport.xlsx"

In [ ]:
gp = GetPaths()

In [ ]:
# Set variables
conn_file="C:/Users/Hutto/AppData/Roaming/Esri/Desktop10.7/ArcCatalog/Connection to nrwugspgressp.sde"
#arcpy.CreateVersion_management(conn_file, parent_version="sde.DEFAULT",
 #                                          version_name="chemtest", access_permission="PUBLIC")


folderName = "C:/Users/Hutto/AppData/Roaming/Esri/Desktop10.7/ArcCatalog/"


fileName = "Connection to nrwugspgressp.sde"
serverName = cp.server
serviceName = cp.instance.split(':')[-1]
databaseName = cp.database
authType = "DATABASE_AUTH"
username = cp.user
password = "toolbox"
saveUserInfo = "SAVE_USERNAME"
versionName = 'paulinkenbrandt.chemtest'
saveVersionInfo = "SAVE_VERSION"


# Process: Use the CreateArcSDEConnectionFile function
arcpy.CreateArcSDEConnectionFile_management(folderName,
                                             fileName,
                                             serverName,
                                             serviceName,
                                             databaseName,
                                             authType,
                                             username,
                                             password,
                                             saveUserInfo,
                                             versionName,
                                             saveVersionInfo)

In [ ]:
cp.instance.split(':')[-1]

In [ ]:
workspace = os.path.join(temp, file +".sde")
# Set the workspace environment
env.workspace = workspace


At the end of your script:
arcpy.Delete(workspace)

In [ ]:
conn_file="C:/Users/Hutto/AppData/Roaming/Esri/Desktop10.7/ArcCatalog/Connection to nrwugspgressp.sde"
arcpy.CreateVersion_management(conn_file, parent_version="sde.DEFAULT",
                                           version_name="chemtest", access_permission="PUBLIC")

# self.enviro = conn_file
arcpy.env.workspace = conn_file
desc = arcpy.Describe(conn_file)
cp = desc.connectionProperties
print("\nDatabase Connection Properties:")
print("%-12s %s" % ("  Server:", cp.server))
print("%-12s %s" % ("  Instance:", cp.instance))
print("%-12s %s" % ("  Database:", cp.database))
print("%-12s %s" % ("  User:", cp.user))
print("%-12s %s" % ("  Version:", cp.version))

In [ ]:
print("%-24s %s" % ("Connection String:", desc.connectionString))
print("%-24s %s" % ("WorkspaceFactoryProgID:", desc.workspaceFactoryProgID))
print("%-24s %s" % ("Workspace Type:", desc.workspaceType))

In [ ]:
gp.compare_sde_wqx('ResultsExport.xlsx')

In [ ]:
class GetPaths(object):
    def __init__(self, versionname='chemchild',
                 conn_file="C:/Users/Hutto/AppData/Roaming/Esri/Desktop10.7/ArcCatalog/Connection to nrwugspgressp.sde"):
        # self.enviro = conn_file
        self.chemistry_table = "ugsgwp.sde.ugs_ngwmn_monitoring_phy_chem_results"
        self.activities_table = "ugsgwp.sde.ugs_ngwmn_monitoring_phy_chem_activities"
        self.stations_table = "ugsgwp.sde.ugs_ngwmn_monitoring_locations"
        arcpy.env.workspace = conn_file
        self.enviro = arcpy.env.workspace
        desc = arcpy.Describe(self.enviro)
        self.user = desc.connectionProperties.user
        self.version = versionname
        if f"{self.user}.{versionname}" in [i.name for i in arcpy.da.ListVersions(self.enviro)]:
            arcpy.DeleteVersion_management(self.enviro, f"{self.user}.{versionname}")
            print(f"{self.user}.{versionname} deleted")
            arcpy.CreateVersion_management(self.enviro, parent_version="sde.DEFAULT",
                                           version_name=versionname, access_permission="PUBLIC")
            print(f"{self.user}.{versionname} created")

        else:
            arcpy.CreateVersion_management(self.enviro, parent_version="sde.DEFAULT",
                                           version_name=versionname, access_permission="PUBLIC")
            print(f"{self.user}.{versionname} created")

    def compare_sde_wqx(self, wqx_results_filename, table_type='chem', recon=False):
        """
        compares unique rows in an SDE chem table to that of a WQX download
        :param wqx_results_filename: excel file with wqx results download
        :param enviro: file location or sde connection file location of table
        :param chem_table_name: table with chemistry data in SDE
        :return:
        """
        if table_type == 'chem':
            table_name = self.chemistry_table
            
        else:
            table_name = self.stations_table

        wqx_table = pd.read_excel(wqx_results_filename)
        
        if arcpy.Exists('db_table'):
            arcpy.Delete_management('db_table')
        else:
            pass
        
        arcpy.MakeTableView_management(table_name, 'db_table')
        arcpy.ChangeVersion_management('db_table', 'TRANSACTIONAL', f"{self.user}.{self.version}")

        #sde_table = self.table_to_pandas_dataframe(table_name)
        sde_table = self.table_to_pd_df(table_name)

        if table_type == 'chem':
            wqx_table['uniqueid'] = wqx_table[
                ['Monitoring Location ID', 'Activity ID', 'Characteristic']].apply(
                lambda x: "{:}-{:}-{:}".format(str(x[0]), str(x[1]), x[2]), 1)
            sde_table['uniqueid'] = sde_table[
                ['monitoringlocationid', 'activityid', 'characteristicname']].apply(
                lambda x: "{:}-{:}-{:}".format(str(x[0]), str(x[1]), x[2]), 1)
            wqx_table = wqx_table.drop_duplicates(subset='uniqueid')
            wqx_table = wqx_table.reset_index().set_index('uniqueid')
            sde_table = sde_table.reset_index().set_index('uniqueid')

        else:
            sde_table = sde_table.reset_index().set_index('locationid')
            wqx_table = wqx_table.reset_index().set_index('Monitoring Location ID')

        objtable = []

        for ind in sde_table.index:
            if ind in wqx_table.index:
                objtable.append(sde_table.loc[ind, 'objectid'])

                
        with arcpy.da.Editor(self.enviro) as edit:
            # iterate input table
            with arcpy.da.UpdateCursor('db_table', ['OID@', 'inwqx']) as tcurs:
                for row in tcurs:
                    # index location row[2]=SHAPE@X and row[1]=SHAPE@Y that matches index locations in dictionary
                    if row[0] in objtable:
                        row[1] = 1
                        tcurs.updateRow(row)
                    
        if recon:
            arcpy.ReconcileVersions_management(self.enviro, "ALL_VERSIONS", "sde.SAFETY",
                                               f"{self.user}.{versionname}", "LOCK_ACQUIRED",
                                               "ABORT", "BY_OBJECT", "FAVOR_TARGET_VERSION",
                                               "POST", "DELETE_VERSION", "RecLog.txt")

        #sde_table = self.table_to_pd_df(table_name)
        return wqx_table, sde_table

    def table_to_pandas_dataframe(self, table, field_names=None, query=None, sql_sn=(None, None)):
        """
        Load data into a Pandas Data Frame for subsequent analysis.
        :param table: Table readable by ArcGIS.
        :param field_names: List of fields.
        :param query: SQL query to limit results
        :param sql_sn: sort fields for sql; see http://pro.arcgis.com/en/pro-app/arcpy/functions/searchcursor.htm
        :return: Pandas DataFrame object.
        """
        # TODO Make fast with SQL
        # if field names are not specified
        if not field_names:
            field_names = self.get_field_names(table)
        # create a pandas data frame
        df = pd.DataFrame(columns=field_names)

        # use a search cursor to iterate rows
        with arcpy.da.SearchCursor(table, field_names, query, sql_clause=sql_sn) as search_cursor:
            # iterate the rows
            for row in search_cursor:
                # combine the field names and row items together, and append them
                df = df.append(dict(zip(field_names, row)), ignore_index=True)

        # return the pandas data frame
        return df

    def get_field_names(self, table):
        read_descr = arcpy.Describe(table)
        field_names = []
        for field in read_descr.fields:
            field_names.append(field.name)
        # field_names.remove('OBJECTID')
        return field_names
    
    def table_to_pd_df(self, table, field_names=None, query=None, sql_sn=(None, None)):
        try:
            egdb_conn = arcpy.ArcSDESQLExecute(self.enviro)
            egdb_conn.startTransaction()
            # Perform the query
            egdb_return = egdb_conn.execute(f"SELECT * FROM {table}")
            egdb_conn.commitTransaction()
            return pd.DataFrame(egdb_return,columns = self.get_field_names(table))
            del egdb_conn
        except Exception as err:
            print(err)


In [ ]:


import ipywidgets as widgets
slider = widgets.FloatSlider(description='x')
text = widgets.FloatText(disabled=True, description='$x^2$')
text.disabled
def compute(*ignore):
    text.value = str(slider.value**2)
slider.observe(compute, 'value')
slider.value = 4
widgets.VBox([slider, text])



# Stations

Compare station export from the CDX to the data in the ArcGIS SDE table.

In [10]:
wqx_stat_table, sde_stat_table = wqxsde.compare_sde_wqx(wqx_stations_filename,
                                                        sde.enviro, 
                                                        sde.stations_table,
                                                        table_type="stat")

AttributeError: module 'wqxsde' has no attribute 'compare_sde_wqx'

Get data from SDE and prepare for WQX import configuration number 6441 <br>
https://cdx.epa.gov/WQXWeb/ImportConfigurationDetail.aspx?mode=import&impcfg_uid=6441

In [ ]:
stats = wqxsde.SDEStationstoWQX(sde_stat_table,"G:/My Drive/Python/Pycharm/wqxsde")
stats.save_file() # save file to csv


In [ ]:
sde_stats = stats.sde_stat_import

# Chemistry

In [ ]:
wqx_results_filename = wqxpath + "WQXResultsExport.xlsx"

In [ ]:
sdechem = wqxsde.table_to_pandas_dataframe(sde.chemistry_table)

In [ ]:
sdechem.columns

In [ ]:
wqx_chem_table, sde_chem_table = wqxsde.compare_sde_wqx(wqx_results_filename,
                                                        sde.enviro, 
                                                        sde.chemistry_table,
                                                        table_type="chem")

## EPA Lab Data to SDE

In [ ]:
epa_file = "G:/Team Drives/UGS_Groundwater/Projects/NGWMN/Chemistry Data/EPAdata_results/2018/1807003 FINAL EPA Excel 12 Sep 18 1357.xls"
schema_file_path = "G:/My Drive/Python/Pycharm/wqxsde/examples/Domains.xlsx"
save_path = "G:/My Drive/WORK/Groundwater Chemistry/"

for file in glob.glob(
        "G:/Team Drives/UGS_Groundwater/Projects/NGWMN/Chemistry Data/EPAdata_results/2018/*.xls"
):

    epa = wqxsde.ProcessEPASheet(file, save_path, schema_file_path)

    epa.append_data()

In [ ]:
epa.append_data()

In [ ]:
{'Ammonia as N':['Ammonia','as N'], 
 'Sulfate as SO4':['Sulfate','as SO4'], 
 'Nitrate as N':['Nitrate','as N'], 
 'Nitrite as N':['Nitrite','as N'], 
 'Orthophosphate as P':['Orthophosphate','as P']}


In [ ]:
epa_file = "G:/Team Drives/UGS_Groundwater/Projects/NGWMN/Chemistry Data/EPAdata_results/2018/1807003 FINAL EPA Excel 12 Sep 18 1357.xls"

epa_raw_data = pd.read_excel(epa_file)

epa_rename = {'Laboratory':'LaboratoryName',
              'LabNumber':'ActivityID',
              'SampleName':'MonitoringLocationID',
              'Method':'ResultAnalyticalMethodID',
              'Analyte':'CharacteristicName',
              'ReportLimit':'ResultDetecQuantLimitUnit',
              'Result':'ResultValue',
              'AnalyteQual':'ResultQualifier',
              'AnalysisClass':'ResultSampleFraction',
              'ReportLimit':'ResultDetecQuantLimitMeasure'
             }

epa_raw_data = epa_raw_data.rename(columns=epa_rename)

epa_raw_data['ResultSampleFraction'] = epa_raw_data['ResultSampleFraction'].apply(lambda x: 'Total' if 'WET' else x, 1)

def hasless(df):
    if '<' in str(df['ResultValue']):
        df['ResultValue'] = None
        df['ResultDetectionCondition'] = 'Below Reporting Limit'
        df['ResultDetectQuantLimitType'] = 'Lower Reporting Limit'
    elif '>' in str(df['ResultValue']):
        df['ResultValue'] = None
        df['ResultDetectionCondition'] = 'Above Reporting Limit'
        df['ResultDetectQuantLimitType'] = 'Upper Reporting Limit'
    else:
        df['ResultValue'] = pd.to_numeric(df['ResultValue'])
    return df

epa_raw_data = epa_raw_data.apply(lambda x: hasless(x),1)


def resqual(x):
    if pd.isna(x[1]) and x[0] == 'Below Reporting Limit':
        return 'BRL'
    elif pd.notnull(x[1]):
        return x[1]
    else:
        return None
    

def filtmeth(x):
    if "EPA" in x:
        x = x.split(' ')[1]
    elif '/' in x:
        x = x.split('/')[0]
    else:
        x = x
    return x


epa_raw_data['ResultAnalyticalMethodID'] = epa_raw_data['ResultAnalyticalMethodID'].apply(lambda x: filtmeth(x), 1)
epa_raw_data['ResultAnalyticalMethodContext'] = 'USEPA'
epa_raw_data['ProjectID'] = 'UNGWMN'
epa_raw_data['ResultQualifier'] = epa_raw_data[['ResultDetectionCondition',
                                                'ResultQualifier']].apply(lambda x: resqual(x),1)

epa_raw_data['Sample Date'] = epa_raw_data['Sampled'].apply(lambda x: "{:%Y-%m-%d}".format(x),1)
epa_raw_data['Sample Time'] = epa_raw_data['Sampled'].apply(lambda x: "{:%H:%M}".format(x),1)
epa_raw_data['AnalysisStartDate'] = epa_raw_data['Analyzed'].apply(lambda x: "{:%Y-%m-%d}".format(x),1)

epa_drop = ['Batch','Analysis','Analyst','CASNumber','Elevation','LabQual',
            'Client','ClientMatrix','Dilution','SpkAmt','UpperLimit','Recovery',
           'Surrogate','LowerLimit','Latitude','Longitude','SampleID','ProjectNumber',
            'Sampled','Analyzed','PrepMethod','Prepped','Project']
epa_data = epa_raw_data.drop(epa_drop,axis=1)
epa_data

In [ ]:
sdechem.columns

In [ ]:
sdeact.columns

In [ ]:
epa_raw_data['ResultQualifier']

## State Lab Chemistry Import to SDE

### Compile Old State Lab Data

In [13]:
state_lab_results = "G:/Shared drives/UGS_Groundwater/Projects/NGWMN/Chemistry Data/StateLab/UTGS_EDD_20191115.txt"
state_lab = pd.read_csv(state_lab_results, sep='\t')


In [35]:
import glob
import os
dfs = {}
cnt = 0
for file in glob.glob("G:/Shared drives/UGS_Groundwater/Projects/NGWMN/Chemistry Data/StateLab/UTGS_EDD_2*.txt"):
#    print(file)
    fname = os.path.basename(file)[:-4]
    if fname != 'UTGS_EDD_20161117':
        dfs[fname] = pd.read_csv(file,sep='\t')
        cntn = len(dfs[fname])
        cnt += cntn
        print(cnt)
        #dfs[fname].columns = [i.strip().title() for i in dfs[fname].columns]
        #dfs[fname] = dfs[fname].sort_index(axis=1)
df = pd.concat(dfs,axis=0,ignore_index=True)


811
2885
4388
6871


In [36]:
sdewqx.ugs_tabs['Station'].set_index(['locationid']).loc['390714112200401']

objectid                                                      198
locationname                                    (C-19- 4)31ada- 1
locationtype                                                 Well
locationdesc                                              67-3499
huc8                                                     16030005
                                      ...                        
personnel                                                    None
inwqx                                                           0
shape           280000000100000001000400206900000C000000010000...
stickup                                                       NaN
globalid                   {B7E92F5A-880C-42C8-9FD0-1F2D16B01B8E}
Name: 390714112200401, Length: 63, dtype: object

In [37]:
import folium

station = sdewqx.ugs_tabs['Station'].set_index(['locationid']).loc['384124109393101']
m = folium.Map(location=[station.latitude,station.longitude])
folium.Marker([station.latitude,station.longitude], popup=f'{station.locationname}').add_to(m)
m

In [38]:
import re

for key, value in matches.items():
    fd = key+f"|{value}"
    df.loc[df['Sample Description'].str.contains(f'(?i){fd}'),'Station ID'] = value

df#def findWholeWord(w):
#    return re.compile(r'\b({0})\b'.format(w), flags=re.IGNORECASE).search
#print(re.compile('(foo(.*?)bar)', flags=re.IGNORECASE).search('FOO da da BAR'))

C:\Users\Hutto\Anaconda3\envs\geo37\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,Project Name,Sample Number,Station ID,Sample Date,Sample Time,Sample Type,Cost Code,Agency Bill Code,Trip ID,Sample Description,...,Sample Detect Limit,Sample Report Limit,Units,Dilution Factor,Batch Number,Analysis Date,Project Comment,Sample Comment,Test Comment,Result Comment
0,C2019-06582,2221499,390714112200401,11/20/2019 12:00:00 AM,14:00,NaN,350,27,NaN,LRPX UWR4 LOWER ROCKY PASS SPRING PIPE,...,0.083,2.0,UMHOS-CM,1.0,EPA120.1-20191204-1,12/04/2019 12:00:00 AM,NaN,NaN,NaN,NaN
1,C2019-06582,2221499,390714112200401,11/20/2019 12:00:00 AM,14:00,NaN,350,27,NaN,LRPX UWR4 LOWER ROCKY PASS SPRING PIPE,...,11.700,20.0,MG-L,1.0,EPA160.1-20191202-2,11/27/2019 12:00:00 AM,NaN,NaN,NaN,NaN
2,C2019-06582,2221499,390714112200401,11/20/2019 12:00:00 AM,14:00,NaN,350,27,NaN,LRPX UWR4 LOWER ROCKY PASS SPRING PIPE,...,1.490,4.0,MG-L,1.0,EPA160.2-20191202-1,11/27/2019 12:00:00 AM,NaN,NaN,NaN,NaN
3,C2019-06582,2221499,390714112200401,11/20/2019 12:00:00 AM,14:00,NaN,350,27,NaN,LRPX UWR4 LOWER ROCKY PASS SPRING PIPE,...,0.096,0.1,NTU,1.0,EPA180.1-20191127-1,11/27/2019 12:00:00 AM,NaN,NaN,NaN,NaN
4,C2019-06582,2221499,390714112200401,11/20/2019 12:00:00 AM,14:00,NaN,350,27,NaN,LRPX UWR4 LOWER ROCKY PASS SPRING PIPE,...,0.440,3.5,MG-L,1.0,EPA325.2-20191202-1,12/02/2019 12:00:00 AM,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6866,C2019-05927,2215282,390714112200401,10/20/2019 12:00:00 AM,15:40,NaN,350,27,NaN,06W CASTLE VALLEY ADELE CREEKSIDE,...,2.000,4.0,MG-L,4.0,EPA200.8_M-20191023-2,10/23/2019 12:00:00 AM,NaN,NaN,NaN,NaN
6867,C2019-05927,2215282,390714112200401,10/20/2019 12:00:00 AM,15:40,NaN,350,27,NaN,06W CASTLE VALLEY ADELE CREEKSIDE,...,2.000,4.0,MG-L,4.0,EPA200.8_M-20191023-2,10/23/2019 12:00:00 AM,NaN,NaN,NaN,NaN
6868,C2019-05927,2215282,390714112200401,10/20/2019 12:00:00 AM,15:40,NaN,350,27,NaN,06W CASTLE VALLEY ADELE CREEKSIDE,...,2.000,4.0,MG-L,4.0,EPA200.8_M-20191023-2,10/23/2019 12:00:00 AM,NaN,NaN,NaN,NaN
6869,C2019-05927,2215282,390714112200401,10/20/2019 12:00:00 AM,15:40,NaN,350,27,NaN,06W CASTLE VALLEY ADELE CREEKSIDE,...,2.000,4.0,MG-L,4.0,EPA200.8_M-20191023-2,10/23/2019 12:00:00 AM,NaN,NaN,NaN,NaN


In [18]:
state_lab2 = pd.read_csv(r"G:\My Drive\Python\Pycharm\wqxsde\examples\id_match.csv")
state_lab2['stationid'] = state_lab2['Sample Description'].apply(lambda x: matches)

In [40]:
df.columns

Index(['Project Name', 'Sample Number', 'Station ID', 'Sample Date',
       'Sample Time', 'Sample Type', 'Cost Code', 'Agency Bill Code',
       'Trip ID', 'Sample Description', 'Collector', 'Sample Received Date',
       'Chain of Custody', 'Replicate Number', 'Test Number', 'Method Agency',
       'Method ID', 'Method Description', 'Param Description', 'CAS Number',
       'Matrix Description', 'Problem Identifier', 'Result Code',
       'Result Value', 'Lower Report Limit', 'Method Detect Limit',
       'Sample Detect Limit', 'Sample Report Limit', 'Units',
       'Dilution Factor', 'Batch Number', 'Analysis Date', 'Project Comment',
       'Sample Comment', 'Test Comment', 'Result Comment'],
      dtype='object')

In [17]:
matches = {'SPRINGVILLE':'401043111361801',
           '(C-19-.*?4)31ada-.*?1':'390714112200401',
           'AM.*?FK|FORK.*?WELL|wl':'402105111472601', 
           'Cedar.*?V.*?Fur':'401656112020301',
           'FAIRFIELD':'401539112045501', 
           'clover.*?sp':'402050112330201',
           'Pecan':'370858113220301',
           'delle.*?sp':'403328112442201', 
           'sp.*?timpie|timpie.*?sp':'404425112384801',
           'sp.*?simpson|simpson.*?sp':'400204112465801', 
           'sp.*?antelope|antelope.*?sp':'382238113205301',
           'sp.*?willow.*?!piez|willow.*?sp.*?!piez|404975112573501':'404975112573501', 
           'PLACER|OPEN.*?LAND':'383709109230701',
           'BAILEY|KEELER':'383832109243901', 
           'STUCKI':'383922109253801',
           'Jenks|ADELE|CREEKSIDE':'383854109242901', 
           'PORCUPINE':'383453109200601',
           'LOOP.*?ROAD':'383746109214001', 
           'GOLF.*?MOAB|MOAB.*?GOLF|383210109285801':'383210109285801',
           'COURTHOUSE.*?WASH':'384113109391001', 
           'PRICE.*?GOLF':'393841110514801',
           'NEPHI.*?WELL':'394313111504701', 
           'WL.*?FLOWING|FLOWING.*?WL|FLOWING.*?WELL':'3959531115425101',
           'SP.*?GOSHEN.*?WARM|GOSHEN.*?WARM.*?SP|395717111512301':'395717111512301',
           'WL.*?ELEMEN|ELEMEN.*?WL|ELEMEN.*?WELL':'404724111562501',
           'GRATEFUL':'404508112522401', 
           'RHONDA|EASY.*?ST':'383040109281201',
           'LRP1A|LOWER.*?R.*?CKY.*?PASS.*?PIEZ':'413308113492701',
           'WLP1C|WILLOW.*?SP.*?PIEZOMETER.*?1C':'413435113475901',
           'WLP1B|WILLOW.*?SP.*?PIEZOMETER.*?1B':'413435113475801',
           'BRYCE|BC26W.*?RICH':'374159112121001',
           'BRYCE.*?BC13W.*?POE':'374124112135501',
           'BRYCE.*?BC22W.*?RUBY.*?3':'374138112103102', 
           'BRYCE.*?BC21W.*?RUBY.*?2':'374138112103101',
           'BRYCE.*?BC15S.*?LOWER.*?BERRY':'374410112133001', 
           'BRYCE.*?BC27W.*?AIRPORT':'374236112092501',
           'BRYCE.*?BC40W.*?ELK':'374232112100501',
           'BRYCE.*?BC44W.*?NPS.*?1':'373755112124301', 
           'BRYCE.*?BC17S.*?UPPER.*?BERRY':'374548112142201',
           'BRYCE.*?BC25W.*?UDOT':'374156112112401',
           'BRYCE.*?BC51W.*?BRISTLE.*?CONE':'374646112063201',
           'BRYCE.*?BC11S.*?NPS.*?4':'373801112124701',
           'BRYCE.*?BC2S.*?TROPIC.*?1':'373639112152101',
           'BRYCE.*?BC4S.*?TROPIC.*?2':'373629112151801',
           'BRYCE.*?BC12W USFS':'374017112130801',
           'BRYCE.*?BC53S.*?WATER':'373534112151101',
           'BRYCE.*?BC6S.*?DRIPPING.*?VAT':'374500112023001',
           'BRYCE.*?BC48W.*?SITLA':'374438112080601',
           'HAMMOND':'391159111543401',
           'NINE.*?MILE':'391020111421301', 
           'OLSEN':'392335111354601',
           'WL.*?T34.*?190605':'394809112161101',
           'SP.*?MUD2.*?190605':'394800112160201',
           'WL.*?T2.*?190605':'395215112140501',
           'SP.*?MUD1.*?190605':'395214112140301',
           'WL.*?T53.*?190605':'395059112120501',
           'ST.*?T15.*?190605':'395051112120101',
           'ST.*?T8.*?190605':'395126112124501',
           'BC35S.*?BRYCE|MOSSY.*?CAVE':'373949112065401',
           'BC28W.*?BRYCE.*?1':'374750112043401',
           'BC30W.*?BRYCE.*?LANDFILL.*?3':'374759112040101',
           'BC29W.*?BRYCE.*?LANDFILL.*?2':'374801112042101',
           'BC31S.*?BRYCE.*?TOM.*?BEST':'374855112054701',
           'BC19W.*?BRYCE.*?KINGS.*?CG':'373647112154301',
           'TRI.*?R.*?NCH':'395718112234301',
           'SP.*?HUNGTINGTON|HUNTING.*?':'392025110565901', 
           'IRR.*?WELL.*?GROUSE|GROUSE.*?IRR.*?WELL':'413332113545101',
           'IRR.*?WELL.*?13':'415657112514101',
           'IRR.*?WELL.*?5':'415906112485001',
           '67.*?3499|':'390714112200401',
           'FLOWELL.*?WELL|WL.*?FLOWELL':'385822112265201',
           'BLACK.*?SPRING':'384610112495201',
           'ASHELY.*?GORGE':'403429109370601',
           'WHITEROCKS.*?SP|SP.*?WHITEROCKS':'402906109572401',
           'SG21C':'395312113244803', 
           'MUD.*?1':'395214112140301', 
           'MUD.*?2':'395214112140301', 
           'T8':'395132112124901', 
           'T15':'395051112120101', 
           'T53':'395059112120501', 
           'T34':'394809112161101', 
           'T2':'395215112140501',
           'KGSP.*?!PIEZ|KEG.*?SPR.*?!PIEZ':'413507113472401', 
           'NORTH.*?BEDKE.*?SPR':'413810113493901',
           'BC7W.*?RUBY.*?4':'374116112083901',
           'BC51W.*?BRYCE|BRISTLE.*?CONE':'374646112063201', 
           'AGI3C':'385630114020202', 
           'PORCUPINE':'383453109200601'
          }

In [ ]:
matches = {'SPRINGVILLE FISH':'401043111361801',
           'NGWMN-AM. FK. PRIVATE WELL':'402105111472601', 
           'NGWMN-CEDAR V FUR FARM':'401656112020301',
           'NGWMN-FAIRFIELD WELL':'401539112045501', 
           'NGWMN-CLOVER SPRINGS':'402050112330201',
           'NGWMN-DELLE SPRINGS':'403328112442201', 
           'NGWMN-BIG TIMPIE SPRING':'404425112384801',
           'NGWMN-SP-SIMPSON-19':'400204112465801', 
           'NGWMN-SP-ANTELOPE-19':'382238113205301',
           'NGWMN-SP-WILLOW-19':'404975112573501', 
           'WL01 CASTLE VALLEY OPEN LANDS PLACER':'383709109230701',
           'WL02 CASTLE VALLEY BAILEY':'383832109243901', 
           'WL03 CASTLE VALLEY STUCKI':'383922109253801',
           'WL04 CASTLE VALLEY JENKS CASTLE':'383854109242901', 
           'WL05 CASTLE VALLEY PORCUPINE':'383453109200601',
           'WL06 CASTLE VALLEY LOOP ROAD':'383746109214001', 
           'WL1 NGWMN GOLF COURSE MOAB':'383210109285801',
           'SP3 NGWMN COURTHOUSE WASH':'384113109391001', 
           'SP04 NGWMN PRICE GOLF':'393841110514801',
           'NGWMN-WL-NEPHIWELL2019':'394313111504701', 
           'NGWMN-WL-FLOWING1-2019':'3959531115425101',
           'NGWMN-SP-GOSHENWARM-2019':'395717111512301',
           'NGWMN-WL-ELEMENTARYWELL-2019':'404724111562501',
           'NGWMN-WL-GRATEFULTOMATO-2019':'404508112522401', 
           'WL02 NGWMN RHONDA EASY ST':'383040109281201',
           'LRP1A LOWER ROCKY PASS SPRING PIEZOMETER':'413308113492701',
           'WLP1C WILLOW SPRING CANYON PIEZOMETER 1C':'413435113475901',
           'WLP1B WILLOW SPRING CANYON PIEZOMETER 1B':'413435113475801',
           'BRYCE CANYON BC26W RICH':'374159112121001',
           'BRYCE CANYON BC13W POE':'374124112135501',
           'BRYCE CANYON BC22W RUBY 3':'374138112103102', 
           'BRYCE CANYON BC21W RUBY 2':'374138112103101',
           'BRYCE CANYON BC15S LOWER BERRY':'374410112133001', 
           'BRYCE CANYON BC27W AIRPORT':'374236112092501',
           'BRYCE CANYON BC40W ELK':'374232112100501',
           'BRYCE CANYON BC44W NPS 1':'373755112124301', 
           'BRYCE CANYON BC17S UPPER BERRY':'374548112142201',
           'BRYCE CANYON BC25W UDOT':'374156112112401',
           'BRYCE CANYON BC51W BRISTLE CONE':'374646112063201',
           'BRYCE CANYON BC11S NPS 4':'373801112124701',
           'BRYCE CANYON BC2S TROPIC 1':'373639112152101',
           'BRYCE CANYON BC4S TROPIC 2':'373629112151801',
           'BRYCE CANYON BC12W USFS':'374017112130801',
           'BRYCE CANYON BC53S WATER STOP':'373534112151101',
           'BRYCE CANYON BC6S DRIPPING VAT':'374500112023001',
           'BRYCE CANYON BC48W SITLA':'374438112080601',
           'NGWMN WL 01 HAMMOND':'391159111543401',
           'NGWMN SP 02 NINE MILE':'391020111421301', 
           'NGWMN WL 03 OLSEN':'392335111354601',
           'WL-T34-190605':'394809112161101',
           'SP-MUD2-190605':'394800112160201',
           'WL-T2-190605':'395215112140501',
           'SP-MUD1-190605':'395214112140301',
           'WL-T53-190605':'395059112120501',
           'ST-T15-190605':'395051112120101',
           'ST-T8-190605':'395126112124501',
           'BC35S BRYCE MOSSY CAVE':'373949112065401',
           'BC28W BRYCE CANYON LANDFILL 1':'374750112043401',
           'BC30W BRYCE CANYON LANDFILL 3':'374759112040101',
           'BC29W BRYCE LANDFILL 2':'374801112042101',
           'BC31S BRYCE CANYON TOM BEST SPG':'374855112054701',
           'BC19W BRYCE CANYON KINGS CG':'373647112154301',
           'NGWMN-TRIANGLE RANCH SPRING':'395718112234301',
           'NGWMN SP HUNGTINGTON':'392025110565901', 
           'NGWMN IRRIGATION WELL GROUSE CREEK':'413332113545101',
           'NGWMN IRRIGATION WELL 13':'415657112514101',
           'NGWMN IRRIGATION WELL #5':'415906112485001',
           'NGWMN - WL-67-3499-2019':'390714112200401',
           'NGWMN - WL-FLOWELL WELL-2019':'385822112265201',
           'NGWMN - SP-BLACKSPRING-2019':'384610112495201',
           'NGWMN - WL-ASHELY GORGE-2019':'403429109370601',
           'NGWMN - SP-WHITEROCKS-2019':'402906109572401',
           'SG21C UGMN SG21C':'395312113244803', 
           'MUD 1':'395214112140301', 
           'MUD 2':'395214112140301', 
           'T8':'395132112124901', 
           'T15':'395051112120101', 
           'T53':'395059112120501', 
           'T34':'394809112161101', 
           'T2':'395215112140501',
           'KGSP UWR4 KEG SPRING':'413507113472401', 
           'NNBS UWR4 NORTH NORTH BEDKE SPRING':'413810113493901',
           'BC7W BRYCE CANYON RUBY 4':'374116112083901',
           'BC51W BRYCE CANYON BRISTLE CONE':'374646112063201', 
           'AGI3C NGWMN WL-AGI3C-2019':'385630114020202', 
           '01W CASTLE VALLEY OPEN LANDS PLACER':'383709109230701',
           '02W CASTLE VALLEY KEELER BAILEY':'383832109243901', 
           '03W CASTLE VALLEY STUCKI':'383922109253801',
           '04W CASTLE VALLEY LOOP ROAD':'383746109214001', 
           '05W CASTLE VALLEY PORCUPINE':'383453109200601',
           '06W CASTLE VALLEY ADELE CREEKSIDE':'384124109393101'
          }

In [ ]:
slfiles = "G:/Team Drives/UGS_Groundwater/Projects/NGWMN/Chemistry Data/StateLab/"

old_to_new = {'Sample#Number':'Sample Number',
              'Sample#Date':'Date Collected',
              'Sample#Time':'Time Collected',
              'Analysis#Date':'Analysis Date',
              'CAS#Number':'CAS Number',
              'Agency#Bill#Code':'Agency Bill Code',
              'Batch#Number':'Batch Number',
              'Comment#Text':'Result Comment',
              'Chain#of#Custody#Ind.':'Chain of Custody',
              'Cost#Code':'Cost Code',
              'Dilution#Factor':'Dilution Factor',
              'Lower#Report#Limit':'Lower Report Limit',
              'Matrix#Description':'Matrix Description',
              'Method#Agency':'Method Agency',
              'Method#Description':'Method Description',
              'Method#Detect#Limit':'Method Detect Limit',
              'Param#Description':'Param Description',
              'Problem#Identifier':'Problem Identifier',
              'Method#ID':'Method ID',
              'Sample#Comment':'Sample Comment',
              'Sample#Description':'Sample Description',
              'Result#Value':'Result Value',
              'Sample#Recieved#Date':'Sample Received Date',
              'Replicate#Number':'Replicate Number',
              'Result#Code':'Result#Code',
              'Station#ID':'Station ID',
              'Sample#Type':'Sample Type',
              'Trip#ID':'Trip ID'}

oldfields = ['%#Confidence#Limit', 'Confidence#Limit', 'Comment#Number', 'Comment#Number.1', 'Comment#Text.1',
             'Lab#Code', 'Preparation#Date', 'Source#Code',  'Billing#Code','Matrix#Number',
             'Upper#Quant#Limit','Result#Code','Method#Number','Param#Number']


stlbnew = {}
stlbold = {}

for file in glob.glob(slfiles+"*.txt"):
    
    shrtnm = os.path.splitext(os.path.basename(file))[0]
    if "UTGS_EDD" in shrtnm: 
        stlbnew[shrtnm] = pd.read_csv(file, sep='\t', error_bad_lines=False)
        samecols = {'Collected By':'Collector',
                    'Analyte CAS Number':'CAS Number',
                    'Method Number':'Method ID',
                    'Method Reported Name':'Param Description',
                   'Entry Qualifier':'Problem Identifier',
                    'Result Flags':'Result Code',
                   'Result Entry':'Result Value',
                    'Sample Detection Limit':'Sample Detect Limit',
                    'Date Analyzed':'Analysis Date',
                   'Method Detection Limit':'Method Detect Limit','Project Comments':'Project Comment',
                   'Sample Comments':'Sample Comment','Test Comments':'Test Comment',
                   'Result Comments':'Result Comment','Sample Reporting Limit':'Sample Report Limit',
                   'Result Dilution Factor':'Dilution Factor','Date Recieved':'Sample Received Date',
                   'Specimen Source':'Matrix Description','Customer Internal Code':'Cost Code',
                   'Collecting Agency':'Agency Bill Code',
                   'Result Units':'Units'}
        stlbnew[shrtnm] = stlbnew[shrtnm].rename(columns=samecols)
    else:
        stlbold[shrtnm] = pd.read_csv(file, sep='\t', error_bad_lines=False)
        stlbold[shrtnm] = stlbold[shrtnm].rename(columns=old_to_new)
new_st_data = pd.concat(stlbnew).reset_index().drop(['level_0','level_1'],axis=1)
old_st_data = pd.concat(stlbold).reset_index().drop(['level_0','level_1'],axis=1)
st_data = pd.concat([new_st_data,old_st_data])
st_data = st_data.drop(oldfields, axis=1)


st_data.columns

In [ ]:
st_data.head()

### Designate the file with the id matches.

In [ ]:
lab_based_chem = "https://cdx.epa.gov/WQXWeb/ImportConfigurationDetail.aspx?impcfg_uid=5926"

field_chem = "https://cdx.epa.gov/WQXWeb/ImportConfigurationDetail.aspx?mode=import&impcfg_uid=5931"

In [ ]:
match_file = "G:/My Drive/Python/Pycharm/wqxsde/examples/id_match.csv"
file_path = "G:/My Drive/Python/Pycharm/wqxsde/examples/UTGS_EDD_20190304.txt"
file_path2 = "G:/Team Drives/UGS_Groundwater/Projects/NGWMN/Chemistry Data/StateLab/UTGS_EDD_20170907.txt"
schema_file_path = "G:/My Drive/Python/Pycharm/wqxsde/examples/Domains.xlsx"

In [ ]:
wqx_chem_table, sde_chem_table = compare_sde_wqx(wqx_results_filename,enviro,chem_table_name,table_type="chem")
sde_chem_import = sde_chem_table[sde_chem_table['inwqx']==0]

### Process the state lab text file to import into database

In [ ]:
state = wqxsde.ProcessStateLabText(file_path,"G:/My Drive/WORK/Groundwater Chemistry",
                                   match_file, schema_file_path)


In [ ]:
state = wqxsde.ProcessStateLabText(file_path,"")

In [ ]:
state_lab_chem = state.run_calcs()

In [ ]:
state_lab_chem['resultid'] = state_lab_chem[['ActivityID','CharacteristicName']].apply(lambda x: str(x[0])+ "-" +x[1],1) 

In [ ]:
state_lab_chem.to_csv("G:/My Drive/WORK/Groundwater Chemistry/crosscheck.csv")

### Import SDE Data

In [ ]:
state.append_data()

## Pull SDE Data and Prep for WQX import

In [ ]:
onlineacts

In [ ]:
sdefieldact.head()

In [ ]:
actmatch.keys()

In [ ]:
sdefieldact['ActivityStartDate'] = sdefieldact['ActivityID'].apply(lambda)

### Pull AGOL Field data and transpose

In [ ]:
https://services.arcgis.com/ZzrwjTRez6FJiOq4/arcgis/rest/services/ActivitiesCollection_lite/FeatureServer/1

In [15]:
gis = dbconnect.get_gis()

In [16]:
field_activities = gis.content.get('65fc8d796844452cae34d7d2f9d6c060')

In [28]:
field_raw = pd.read_excel("G:/Shared Drives/UGS_Groundwater/Projects/NGWMN/Chemistry Data/Field_Obs.xlsx",
                         index_col='Activity ID')
df = field_raw[['pH','Temp','Cond']]
dfs = df.stack().reset_index()
dfs.columns = ['Activity ID','Characteristic Name','Result Value']
field = pd.merge(dfs,field_raw, left_on = 'Activity ID', right_index=True)
field.drop(['pH','Temp','Cond'], inplace=True, axis=1)
unitdict = {'pH':'None','Temp':'deg C','Cond':'uS/cm'}
chardict = {'pH':'pH','Temp':'Temperature, water','Cond':'Specific conductance'}
field['Result Unit'] = field['Characteristic Name'].apply(lambda x: unitdict.get(x),1)
field['Characteristic Name'] = field['Characteristic Name'].apply(lambda x: chardict.get(x),1)
field['Activity Type'] = 'Field Msr/Obs'
field['Activity Media Name'] = 'Water'
field['Activity Media Subdivision'] = 'Groundwater'
field['Result Value Type'] = 'Actual'
field['Result Qualifier'] = ''

In [26]:
arcgis.features.FeatureLayerCollection('https://services.arcgis.com/ZzrwjTRez6FJiOq4/arcgis/rest/services/ActivitiesCollection_lite/FeatureServer/1')

Exception: Token Required
Token Required
(Error Code: 499)

In [24]:
field_activities.query('ActivityID is NOT NULL')

AttributeError: 'Item' object has no attribute 'query'

In [ ]:
https://services.arcgis.com/ZzrwjTRez6FJiOq4/ArcGIS/rest/services/ActivitiesCollection_lite/FeatureServer/1?

In [23]:
repsns.url

'https://services.arcgis.com/ZzrwjTRez6FJiOq4/ArcGIS/rest/services/ActivitiesCollection_lite/FeatureServer/1/query?where=ActivityID+is+NOT+NULL&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnDistinctValues=true&outFields=%2A&orderByFields=ActivityID&sqlFormat=standard&f=pjson&token=B0c0oan7KJBxQLTQzdPNi6IHZmQXM4x2BAeatBob1LukJYLY39I6k5FQYI2ZcA_i2ssxwNdzrlNjIEAwH-tJba22OQFCKN9sfqdJhrFzamZPxqx5vd8sZV4qc9KDf0xamLRU9STxR325Rh7YAnO5Dg..'

In [22]:
url = 'https://services.arcgis.com/ZzrwjTRez6FJiOq4/ArcGIS/rest/services/ActivitiesCollection_lite/FeatureServer/1/query?'
kwargs = {}
kwargs['where'] = 'ActivityID is NOT NULL'
kwargs['returnIdsOnly'] = 'false'
kwargs['returnUniqueIdsOnly'] = 'false'
kwargs['returnCountOnly'] ='false'
kwargs['returnDistinctValues'] ='true'
kwargs['outFields'] = '*'
kwargs['orderByFields'] ='ActivityID'
kwargs['sqlFormat'] ='standard'
kwargs['f'] ='pjson'
kwargs['token'] = dbconnect.chem_getter_api()
repsns = requests.get(url, params=kwargs)
onlineacts = pd.DataFrame([i['attributes'] for i in repsns.json()['features']])
onlineacts.head()

afields = ['ActivityID', 'ProjectID', 'MonitoringLocationID', 
               'ActivityStartDate', 'ActivityStartTime']

actmatch = onlineacts[afields].set_index('ActivityID').to_dict()

sdefieldact = onlineacts[['ActivityID','temperature_c','conduct_uscm','pH']].set_index('ActivityID').stack().reset_index()

sdefieldact.columns = ['ActivityID','CharacteristicName','ResultValue']

def funitsandchar(df):
    fcharnms = {'temperature_c':['Temperature, water','deg C'],
                'conduct_uscm':['Specific conductance','uS/cm'],
                'pH':['pH','None']}
    df['ResultUnit'] = fcharnms.get(df['CharacteristicName'])[1]
    df['CharacteristicName'] = fcharnms.get(df['CharacteristicName'])[0]
    return df

sdefieldact = sdefieldact.apply(lambda x: funitsandchar(x),1)
sdefieldact['ActivityID'] = sdefieldact['ActivityID'].apply(lambda x: str(x)+'-FM',1)


sdestat = wqxsde.table_to_pandas_dataframe(sde.stations_table, field_names = statfields)
sdeact = wqxsde.table_to_pandas_dataframe(sde.activities_table, field_names = activfields)
sdechem = wqxsde.table_to_pandas_dataframe(sde.chemistry_table, field_names = chemfields, query = 'inwqx = 0' )


fcharnms = {'temperature_c':['Temperature, water','deg C'],'conduct_uscm':['Specific conductance','uS/cm'],'pH':['pH','None']}

sdefactchem = sdefieldact
#sdefactchem = pd.merge(sdechem, sdefieldact, how = 'inner', on = 'ActivityID')

sdefactchem['Activity Media Name'] = 'Water'
sdefactchem['Activity Start Time Zone'] = 'MDT'
sdefactchem['Activity Type'] = sdefactchem['ActivityID'].apply(lambda x: 'Field Msr/Obs' if '-FM' in x else 'Sample-Routine', 1)
sdefactchem['ResultStatusID'] = 'Final'
sdefactchem['ResultValueType'] = 'Actual'

KeyError: 'features'

### Pull SDE Data and export a WQX csv for import

In [ ]:
chemfields = ['ActivityID', 'ResultDetectionCondition', 'CharacteristicName', 'MethodSpeciation',
              'ResultSampleFraction', 'ResultValue', 'ResultUnit', 'ResultQualifier', 
              'ResultAnalyticalMethodID', 
              'ResultAnalyticalMethodContext', 'LaboratoryName',
              'AnalysisStartDate', 'ResultDetecQuantLimitType',
              'DetecQuantLimitMeasure', 'ResultDetecQuantLimitUnit']

activfields = ['ActivityID', 'ProjectID', 'MonitoringLocationID', 
               'ActivityStartDate', 'ActivityStartTime','temperature_c','conduct_uscm','pH']

statfields = ['LocationID','Send','inwqx']

sdestat = wqxsde.table_to_pandas_dataframe(sde.stations_table, field_names = statfields)
sdeact = wqxsde.table_to_pandas_dataframe(sde.activities_table, field_names = activfields)
sdechem = wqxsde.table_to_pandas_dataframe(sde.chemistry_table, field_names = chemfields, query = 'inwqx = 0' )


fcharnms = {'temperature_c':['Temperature, water','deg C'],'conduct_uscm':['Specific conductance','uS/cm'],'pH':['pH','None']}

sdeactchem = pd.merge(sdechem, sdeact, how = 'inner', on = 'ActivityID')

sdeactchem['Activity Media Name'] = 'Water'
sdeactchem['Activity Start Time Zone'] = 'MDT'
sdeactchem['Activity Type'] = sdeactchem['ActivityID'].apply(lambda x: 'Field Msr/Obs' if '-FM' in x else 'Sample-Routine', 1)
sdeactchem['Sample Collection Equipment Name'] = 'Water Bottle'
sdeactchem['Sample Collection Method ID'] = 'GRAB'
sdeactchem['ResultStatusID'] = 'Final'
sdeactchem['ResultValueType'] = 'Actual'

fieldorder = ['ActivityID', 'Activity Media Name', 'ActivityStartDate', 'ActivityStartTime',
              'Activity Start Time Zone', 'Activity Type', 'AnalysisStartDate', 
              'CharacteristicName', 'LaboratoryName', 'MethodSpeciation',
              'MonitoringLocationID', 'ProjectID', 'ResultAnalyticalMethodContext',
              'ResultAnalyticalMethodID', 'ResultDetectionCondition',
              'DetecQuantLimitMeasure', 'ResultDetecQuantLimitType',
              'ResultDetecQuantLimitUnit', 'ResultQualifier', 'ResultSampleFraction', 
              'ResultStatusID', 'ResultUnit',  'ResultValue', 'ResultValueType',
              'Sample Collection Equipment Name','Sample Collection Method ID']

senders = sdestat[sdestat['Send']=='1']['LocationID'].values


sdeactchemimp = sdeactchem[fieldorder].replace(to_replace='None',value=None)
#sdeactchemimp = sdeactchemimp[sdeactchemimp['Activity Type']=='Sample-Routine']
sdeactchemimp = sdeactchemimp[pd.notna(sdeactchemimp['ProjectID'])]
sdeactchemimp = sdeactchemimp[sdeactchemimp['MonitoringLocationID'].isin(senders)]

save_dir = 'G:/My Drive/Python/Pycharm/wqxsde'
sdeactchemimp.to_csv(save_dir + "/chem_{:%Y%m%d%H%M}.csv".format(pd.datetime.today()), index=False)

In [ ]:
sfieldorder = ['ActivityID','ActivityStartDate', 'ActivityStartTime',
              'CharacteristicName',
              'MonitoringLocationID','ResultAnalyticalMethodContext',
              'ResultAnalyticalMethodID']

sdeactchemimp[sfieldorder][(pd.notnull(sdeactchemimp['ResultAnalyticalMethodContext']))&(pd.isnull(sdeactchemimp['ResultAnalyticalMethodID']))]

In [ ]:
sdeactchemimp

In [ ]:
sdeactchemimp['ProjectID'].unique()

In [ ]:
sdeactchem

In [ ]:
activities = 

In [ ]:
stations[['LocationName','AltLocationID','Offset','BaroLoggerType']][stations['AltLocationID']>1000].to_clipboard()

~~~~sql
CREATE VIEW UGGP.UGGPADMIN.UGS_NGWMN_View_Activities_and_Results
AS
SELECT 
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Activities.ActivityID,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Activities.ActivityStartDate,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Activities.ActivityStartTime,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Activities.ProjectID,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Activities.MonitoringLocationID,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.ResultDetectionCondition,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.CharacteristicName,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.MethodSpeciation,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.ResultSampleFraction,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.ResultValue,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.ResultUnit,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.ResultQualifier,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.ResultStatusID,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.ResultValueType,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.ResultAnalyticalMethodID,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.ResultAnalyticalMethodContext,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.LaboratoryName,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.AnalysisStartDate,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.ResultDetecQuantLimitType,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.ResultDetecQuantLimitMeasure,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.ResultDetecQuantLimitUnit

FROM
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results 
JOIN
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Activities ON
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Activities.ActivityID = UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.ActivityID
~~~~
